In [3]:
import os

In [4]:
%pwd


'c:\\Users\\Krrishna.Jorige\\Desktop\\siri\\braintumor\\research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'c:\\Users\\Krrishna.Jorige\\Desktop\\siri\\braintumor'

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
from Braintumor.constants import *
from Braintumor.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from src.Braintumor.utils import logger
from Braintumor.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-07 09:53:26,363: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-07 09:53:26,366: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-07 09:53:26,367: INFO: common: created directory at: artifacts]
[2024-11-07 09:53:26,368: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-07 09:53:28,388: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7909994
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "34f8fa7dd1c41860ea74dfd86028c78d7ba5eca66bc3d02953ff0ec8986aafdf"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2D83:18C961:86143:8FECC:672CE274
Accept-Ranges: bytes
Date: Thu, 07 Nov 2024 15:53:28 GMT
Via: 1.1 varnish
X-Served-By: cache-dfw-kdal2120033-DFW
X-Cac